In [6]:
import numpy as np # linear algebra
import pandas as pd 
df=pd.read_csv('hacktrain.csv')
df.isnull().sum()

Unnamed: 0       0
ID               0
class            0
20150720_N     560
20150602_N    1200
20150517_N     800
20150501_N     960
20150415_N     480
20150330_N    1120
20150314_N     720
20150226_N    1360
20150210_N     640
20150125_N    1040
20150109_N     880
20141117_N    1280
20141101_N     400
20141016_N    1440
20140930_N     800
20140813_N     560
20140626_N    1600
20140610_N     480
20140525_N     720
20140509_N     880
20140423_N    1760
20140407_N     640
20140322_N    1120
20140218_N    1440
20140202_N     560
20140117_N    1200
20140101_N     400
dtype: int64

In [7]:
df.fillna(df.mean(numeric_only=True), inplace=True)
df.isnull().sum()

Unnamed: 0    0
ID            0
class         0
20150720_N    0
20150602_N    0
20150517_N    0
20150501_N    0
20150415_N    0
20150330_N    0
20150314_N    0
20150226_N    0
20150210_N    0
20150125_N    0
20150109_N    0
20141117_N    0
20141101_N    0
20141016_N    0
20140930_N    0
20140813_N    0
20140626_N    0
20140610_N    0
20140525_N    0
20140509_N    0
20140423_N    0
20140407_N    0
20140322_N    0
20140218_N    0
20140202_N    0
20140117_N    0
20140101_N    0
dtype: int64

In [16]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import accuracy_score


df=pd.read_csv('hacktrain.csv')
# Drop ID column
df.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split into features and target
X = df.drop(columns=['class'])
y = df['class']

# --- Feature Engineering Example (add mean & std of NDVI time series) ---
# You can add more sophisticated features as needed
X['ndvi_mean'] = X.mean(axis=1)
X['ndvi_std'] = X.std(axis=1)






# --- Handle Missing Data ---
imputer = SimpleImputer(strategy='mean')  # or 'median'
X_imputed = imputer.fit_transform(X)

# --- Scale Features ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# --- Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)



custom_weights = {1: 0.217, 0: 1.585, 2:6.796, 3: 1.993, 4: 45, 5: 12.698}


# Apply SMOTE to the training data only
#smote = SMOTE(random_state=42)
#X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


#logistic regression model
model = LogisticRegression(
     C=1,                   # Regularization strength (default is 1.0)
    penalty='l2',            # L2 regularization (default)
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    random_state=42
    #tol=1e-4,
    #class_weight='balanced'   # This line is key!
    #class_weight=custom_weights
)

model.fit(X_train, y_train)


# --- Predict on Test Set ---
y_pred = model.predict(X_test)

# --- Classification Report with All Original Class Labels ---
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))


              precision    recall  f1-score   support

        farm       0.89      0.89      0.89       168
      forest       0.99      1.00      1.00      1232
       grass       0.89      0.64      0.75        39
  impervious       0.85      0.92      0.88       134
     orchard       1.00      0.50      0.67         6
       water       0.89      0.81      0.85        21

    accuracy                           0.97      1600
   macro avg       0.92      0.79      0.84      1600
weighted avg       0.97      0.97      0.97      1600



C:\Users\aarat\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [17]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9663


In [18]:
# --- Load Test Data ---
test_df = pd.read_csv('hacktest.csv')

# --- Prepare Test Data ---
# Save IDs for submission
test_ids = test_df['ID']

# Create test features (same preprocessing as training)
X_test = test_df.drop(columns=['ID'])

# Apply same feature engineering
X_test['ndvi_mean'] = X_test.mean(axis=1)
X_test['ndvi_std'] = X_test.std(axis=1)




# Apply same imputation and scaling (IMPORTANT: use transform, NOT fit_transform)
X_test_imputed = imputer.transform(X_test)  # Using pre-fitted imputer
X_test_scaled = scaler.transform(X_test_imputed)  # Using pre-fitted scaler

# --- Make Predictions ---
predicted_labels = model.predict(X_test_scaled)

# Convert numerical labels back to original class names
predicted_classes = label_encoder.inverse_transform(predicted_labels)

#
# --- Create Submission DataFrame ---
submission = pd.DataFrame({
    'ID': test_ids,
    'class': predicted_classes
})

# --- Save to CSV in the required format ---
submission.to_csv('submission.csv', index=False)

# Optional: Show the first few rows of your submission
print(submission.head(60))


    ID   class
0    1  forest
1    2  forest
2    3  forest
3    4  forest
4    5  forest
5    6  forest
6    7  forest
7    8  forest
8    9  forest
9   10  forest
10  11  forest
11  12  forest
12  13  forest
13  14  forest
14  15  forest
15  16  forest
16  17  forest
17  18  forest
18  19  forest
19  20  forest
20  21  forest
21  22  forest
22  23  forest
23  24   water
24  25   water
25  26   water
26  27   water
27  28  forest
28  29   water
29  30   water
30  31   water
31  32  forest
32  33   water
33  34  forest
34  35   water
35  36   water
36  37   water
37  38   water
38  39   water
39  40   water
40  41   water
41  42   water
42  43   water
43  44   water
44  45   water
45  46  forest
46  47   water
47  48   water
48  49   water
49  50   water
50  51   water
51  52   water
52  53   water
53  54   water
54  55   water
55  56   water
56  57   water
57  58   water
58  59   water
59  60   water


In [94]:
print(label_encoder.classes_)


['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']


In [95]:
import numpy as np
unique_pred = np.unique(y_pred)
print("Predicted classes (encoded):", unique_pred)
print("Predicted classes (original):", label_encoder.inverse_transform(unique_pred))


Predicted classes (encoded): [0 1 2 3 4 5]
Predicted classes (original): ['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']


In [96]:
print("Train class distribution:")
print(y_train.value_counts())
print("Test class distribution:")
print(y_test.value_counts())


Train class distribution:
class
1    4927
0     673
3     535
2     157
5      84
4      24
Name: count, dtype: int64
Test class distribution:
class
1    1232
0     168
3     134
2      39
5      21
4       6
Name: count, dtype: int64


C:\Users\aarat\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\aarat\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\aarat\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\aarat\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in versio

Best C: 1


C:\Users\aarat\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
